## **CNN works on images and below is the example of the consisting layers.**

<img src="./convolutional/convolutional_0.png" alt="convolutional"/>

<img src="./convolutional/convolutional_1.png" alt="convolutional" />
<img src="./convolutional/convolutional_2.png" alt="convolutional" />

## after the convolution layers, there are pooling layers.

![pooling](./convolutional/max_pool.png)

![CNN](./convolutional/convolutional_4.png)

In [35]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F 

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [51]:
#Hyper Parameter
num_epochs = 10
batch_size = 4
learning_rate = 0.001

## Dataset has PILImage images of rgb value [0,255].
## we have to normalize that rgb values.
## <span style = 'color:cyan'>To normalize with pytorch, image array needs to be Tensor </span>

In [38]:
transforming = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [39]:
train_dataset = torchvision.datasets.CIFAR10(root='./dataset',train=True,download=True,transform=transforming)
test_dataset = torchvision.datasets.CIFAR10(root='./dataset',train=False,download=True,transform=transforming)

Files already downloaded and verified
Files already downloaded and verified


# Shape of the dataset is as follows.

In [40]:
print(f'The shape of training dataset is {train_dataset.data.shape} and that of test dataset is {test_dataset.data.shape}')
print(f'Type of data is {type(train_dataset.data)}')

The shape of training dataset is (50000, 32, 32, 3) and that of test dataset is (10000, 32, 32, 3)
Type of data is <class 'numpy.ndarray'>


# TRANSFOMRING OF SINGLE IMAGE

In [41]:
test_img = train_dataset.data[0]
print(type(test_img))
print(test_img.dtype)
print(f'Mean and std before normalize:\nmean = {test_img.mean()}, std = {test_img.std()}')

<class 'numpy.ndarray'>
uint8
Mean and std before normalize:
mean = 103.447265625, std = 51.97371837166704


In [42]:
test_tensor = transforming(test_img)
print(type(test_tensor))
print(f'Mean and std after normalize:\nmean = {test_tensor.mean()}, std = {test_tensor.std()}')

<class 'torch.Tensor'>
Mean and std after normalize:
mean = -0.18864887952804565, std = 0.4077033996582031


In [43]:
#back to pipeline
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

<span style='color:cyan'>kernel_size means filter_size in convolutional layer</span>

In [44]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

# <span style='color:cyan'>Check details of image size reducing down in test_cnn.py</span>

In [45]:
class ConvNet(nn.Module):
    def __init__(self,inputs_channel_size,outputs_channel_size,filter_size,pool_kernel_size):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(in_channels = inputs_channel_size,
                               out_channels = outputs_channel_size, 
                               kernel_size = filter_size)
        self.pool = nn.MaxPool2d(kernel_size = pool_kernel_size)
        self.conv2 = nn.Conv2d(in_channels = outputs_channel_size,
                               out_channels = 16,
                               kernel_size = filter_size)
        self.fc1 = nn.Linear(in_features = 16*5*5,
                             out_features = 120)
        self.fc2 = nn.Linear(in_features = 120,
                             out_features = 84)
        self.fc3 = nn.Linear(in_features = 84,
                             out_features = 10)    
    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)  # tensor flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

## <span style='color:cyan'>The size of image would be smaller after leaving the convlutional layer.</span>

calculation of image size after pooling is as below.

![conv calculation](./convolutional/output_size_after_pooling.png)

original size is (32x32).

filter AKA kernel size is 5.

That's why <span style='color:cyan'>32x32 would become 28x28</span> after first convolutional layer.

In [46]:
in_channel_size = 3 # rgb colors channels
out_channel_size = 6 # anything you want
filter_size = 5
pool_kernel_size = (2,2)

In [52]:
model = ConvNet(inputs_channel_size=in_channel_size,
                outputs_channel_size=out_channel_size,pool_kernel_size=pool_kernel_size,
                filter_size = filter_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [53]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        # watch test_cnn.py  
        # origin shape: [4,3,32,32] = 4,3,1024
        images = images.to(device)
        labels = labels.to(device)
        #forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%2000 == 0:
            print(f'Epoch {epoch+1}/{num_epochs},step {i+1}/{n_total_steps}, loss {loss.item():.3f}')

Epoch 1/10,step 2000/12500, loss 2.362
Epoch 1/10,step 4000/12500, loss 2.302
Epoch 1/10,step 6000/12500, loss 2.297
Epoch 1/10,step 8000/12500, loss 2.276
Epoch 1/10,step 10000/12500, loss 2.293
Epoch 1/10,step 12000/12500, loss 2.254
Epoch 2/10,step 2000/12500, loss 1.790
Epoch 2/10,step 4000/12500, loss 1.713
Epoch 2/10,step 6000/12500, loss 1.931
Epoch 2/10,step 8000/12500, loss 1.333
Epoch 2/10,step 10000/12500, loss 1.828
Epoch 2/10,step 12000/12500, loss 1.701
Epoch 3/10,step 2000/12500, loss 1.985
Epoch 3/10,step 4000/12500, loss 1.139
Epoch 3/10,step 6000/12500, loss 0.691
Epoch 3/10,step 8000/12500, loss 2.052
Epoch 3/10,step 10000/12500, loss 1.847
Epoch 3/10,step 12000/12500, loss 1.286
Epoch 4/10,step 2000/12500, loss 1.077
Epoch 4/10,step 4000/12500, loss 2.243
Epoch 4/10,step 6000/12500, loss 0.777
Epoch 4/10,step 8000/12500, loss 1.768
Epoch 4/10,step 10000/12500, loss 1.585
Epoch 4/10,step 12000/12500, loss 1.576
Epoch 5/10,step 2000/12500, loss 1.084
Epoch 5/10,step 4

In [54]:
with torch.no_grad():
    n_correct = 0
    n_sample = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value,index)
        _,predicted_index = torch.max(outputs,1)
        n_sample += labels.size(0)
        n_correct += (predicted_index==labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted_index[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
        
    acc = 100.0 * n_correct/n_sample
    print(f'Accuracy is {acc}')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i]/n_class_samples[i]
        print(f'Accuracy of {classes[i]}:{acc}')
        

Accuracy is 56.58
Accuracy of plane:59.8
Accuracy of car:68.4
Accuracy of bird:41.5
Accuracy of cat:33.8
Accuracy of deer:35.8
Accuracy of dog:45.1
Accuracy of frog:75.9
Accuracy of horse:68.6
Accuracy of ship:75.8
Accuracy of truck:61.1
